In [2]:
!pip install selenium webdriver-manager pandas beautifulsoup4 lxml

In [3]:
#!pip install selenium webdriver-manager pandas beautifulsoup4 lxml

import time, random, re, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIG ---
BASE = "https://gaez.fao.org/pages/ecocrop-find-plant"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(BASE)
wait = WebDriverWait(driver, 20)

letters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
plants = []

for letter in letters:
    try:
        print(f"🔤 Loading: {letter}")
        # คลิกด้วย JavaScript เพื่อให้มันโหลด iframe จริง ๆ
        driver.execute_script(f"""
            var a = [...document.querySelectorAll('p.letters a')]
                     .find(x => x.textContent.trim() === '{letter}');
            if (a) a.click();
        """)
        time.sleep(3.5 + random.random())

        # รอจนกว่าคำว่า "Found crops" จะปรากฏ (แปลว่าโหลดเสร็จ)
        wait.until(EC.presence_of_element_located((By.XPATH, "//h1[contains(text(), 'Found crops')]")))
        soup = BeautifulSoup(driver.page_source, "lxml")

        # ดึงข้อมูล
        table = soup.select_one("div#content table")
        if not table:
            print(f"⚠️ No table for {letter}")
            continue

        rows = table.select("tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 3:
                name_tag = cols[0].find("a", class_="serviceLink")
                if not name_tag:
                    continue
                name = name_tag.text.strip()
                onclick = name_tag.get("onclick", "")
                m = re.search(r"id=(\d+)", onclick)
                if not m:
                    continue
                pid = m.group(1)
                code = cols[1].text.strip()
                plants.append({
                    "Letter": letter,
                    "Plant Name": name,
                    "Plant Code": code,
                    "URL": f"https://gaez.fao.org/ecocrop/srv/en/cropView?id={pid}"
                })

        print(f"✅ {letter}: {len(plants)} plants so far")
    except Exception as e:
        print(f"❌ {letter}: {e}")
        continue

driver.quit()

df = pd.DataFrame(plants).drop_duplicates(subset=["URL"]).reset_index(drop=True)
df.to_csv("ecocrop_links_final.csv", index=False)
print(f"\nTotal plants scraped: {len(df)}")

🔤 Loading: A
❌ A: Message: 
Stacktrace:
0   chromedriver                        0x00000001033afcf8 cxxbridge1$str$ptr + 2895872
1   chromedriver                        0x00000001033a7c34 cxxbridge1$str$ptr + 2862908
2   chromedriver                        0x0000000102ecd570 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x0000000102f14f34 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 367640
4   chromedriver                        0x0000000102f563d8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 635068
5   chromedriver                        0x0000000102f090f8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 318940
6   chromedriver                        0x000000010337381c cxxbridge1$str$ptr + 2648868
7   chromedriver                        0x0000000103376df8 cxxbridge1$str$ptr + 2662656
8   chromedriver                        0x0000000103354334 cxxbridge1$str

In [5]:
df

""


In [ ]:
# ติดตั้งครั้งแรกหากยังไม่มี
# !pip install selenium webdriver-manager beautifulsoup4 lxml pandas

import re
import time
import random
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

BASE = "https://ecocrop.apps.fao.org"
LIST_TPL = BASE + "/ecocrop/srv/en/cropList?name={letter}&relation=beginsWith"

# ----- ตั้งค่า Chrome -----
opts = webdriver.ChromeOptions()
# เปิด headless ได้หากต้องการรันเบื้องหลัง (เอาคอมเมนต์ออก)
# opts.add_argument("--headless=new")
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--window-size=1366,900")
opts.add_argument("--lang=en-US")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

letters = list("abcdefghijklmnopqrstuvwxyz")
rows_out = []

try:
    for l in letters:
        list_url = LIST_TPL.format(letter=l)
        print(f"🔤 Fetching list: {list_url}")
        driver.get(list_url)

        # รอให้ JS เรนเดอร์เสร็จสั้น ๆ แล้วดึง HTML
        time.sleep(2.2 + random.random())   # ปรับเพิ่มหากเน็ตช้า

        soup = BeautifulSoup(driver.page_source, "lxml")

        # ตารางหลักมักอยู่ใน div#content > table (กันพลาดจึงสำรองเป็น soup.select_one("table"))
        table = soup.select_one("div#content table") or soup.select_one("table")
        if not table:
            print(f"⚠️ No table found for letter {l.upper()}")
            continue

        # เดินตามแถว
        for tr in table.select("tr"):
            tds = tr.find_all("td")
            if len(tds) < 1:
                continue

            # ชื่อพืช: ปกติอยู่คอลัมน์แรก (Name)
            plant_name = tds[0].get_text(strip=True)

            # ปุ่ม View อยู่คอลัมน์ท้าย ๆ แบบ <button onclick="load('/ecocrop/srv/en/cropView?id=289')">View</button>
            # เราจะค้นหา button ที่มี onclick แล้วดึง id ออกมา
            view_btn = tr.select_one("button[onclick*='cropView']")
            if not view_btn:
                # บางหน้ามีลิงก์ <a onclick="load('/ecocrop/srv/en/cropView?id=...')">View</a>
                view_btn = tr.select_one("[onclick*='cropView']")

            if not view_btn:
                # ไม่มีปุ่ม view ก็ข้าม
                continue

            onclick = view_btn.get("onclick", "")
            # จับ id ด้วย regex
            m = re.search(r"cropView\?id=(\d+)", onclick)
            if not m:
                continue

            pid = m.group(1)
            detail_url = f"{BASE}/ecocrop/srv/en/cropView?id={pid}"

            # บางตารางมีคอลัมน์ "Code" หรือ "ID" ถัดจาก Name; เก็บเพิ่มถ้ามี
            code = ""
            if len(tds) >= 2:
                # เดาว่าคอลัมน์ที่ 2 เป็นรหัส ถ้าไม่ใช่ก็จะได้เป็นข้อความในคอลัมน์นั้น (ไม่เสียหาย)
                code = tds[1].get_text(strip=True)

            rows_out.append({
                "Letter": l.upper(),
                "Plant Name": plant_name,
                "Plant Code": code,
                "Plant ID": pid,
                "URL": detail_url
            })

        print(f"✅ {l.upper()}: +{sum(1 for r in rows_out if r['Letter']==l.upper())} items (cumulative: {len(rows_out)})")

        # ถนอมเซิร์ฟเวอร์หน่อย
        time.sleep(0.8 + random.random()*0.6)

finally:
    driver.quit()

# สรุปผลและบันทึก
df = pd.DataFrame(rows_out).drop_duplicates(subset=["URL"]).reset_index(drop=True)
df.to_csv("ecocrop_links_apps.csv", index=False)
print(f"\n🎉 Total items: {len(df)}")
print("Saved -> ecocrop_links_apps.csv")

🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=a&relation=beginsWith
✅ A: +479 items (cumulative: 479)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=b&relation=beginsWith
✅ B: +135 items (cumulative: 614)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=c&relation=beginsWith
✅ C: +332 items (cumulative: 946)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=d&relation=beginsWith
✅ D: +119 items (cumulative: 1065)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=e&relation=beginsWith
✅ E: +210 items (cumulative: 1275)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=f&relation=beginsWith
✅ F: +50 items (cumulative: 1325)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=g&relation=beginsWith
✅ G: +50 items (cumulative: 1375)
🔤 Fetching list: https://ecocrop.apps.fao.org/ecocrop/srv/en/cropList?name=h&rela

In [1]:
# !pip install selenium webdriver-manager beautifulsoup4 lxml pandas

import re, time, random, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

BASE = "https://ecocrop.apps.fao.org"

# ---------- 1) เตรียมรายการพืช ----------
df_links = pd.read_csv("ecocrop_links_apps.csv")  # ต้องมี Plant ID หรือ URL อยู่
if "Plant ID" not in df_links.columns:
    # เผื่อไฟล์มีแค่ URL → ดึง id จาก URL
    df_links["Plant ID"] = df_links["URL"].str.extract(r"id=(\d+)")
df_links = df_links.dropna(subset=["Plant ID"]).copy()
df_links["Plant ID"] = df_links["Plant ID"].astype(str)
df_links["DataSheetURL"] = BASE + "/ecocrop/srv/en/dataSheet?id=" + df_links["Plant ID"]

# ---------- 2) ตั้งค่า Selenium ----------
opts = webdriver.ChromeOptions()
# opts.add_argument("--headless=new")   # เปิดถ้าต้องการรันแบบไม่เปิดหน้าจอ
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--window-size=1366,1000")
opts.add_argument("--lang=en-US")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def parse_ecology_table(soup):
    """คืน dict ของ 3 แถว Ecology ที่ต้องการ: Temperature, Rainfall, Soil PH
       เก็บ Optimal Min/Max และ Absolute Min/Max"""
    # หา table ที่หัวคอลัมน์เขียนว่า Ecology
    target = None
    for tbl in soup.select("div#content table"):
        th = tbl.find("th")
        if th and "Ecology" in th.get_text(strip=True):
            target = tbl
            break
    if target is None:
        return {}

    def pick_row(label):
        # หาแถวที่หัวคอลัมน์ (th) เท่ากับ label
        row = target.find("tr", string=lambda x: False)
        for tr in target.find_all("tr"):
            head = tr.find("th")
            if head and head.get_text(strip=True) == label:
                return tr
        return None

    def get_vals(tr):
        # โครงสร้างคอลัมน์: th | Opt Min | Opt Max | Abs Min | Abs Max | (อีกฝั่งเป็น Soil อื่น ๆ)
        # เราจะอ่านแค่ 4 ช่องแรกหลัง th
        tds = tr.find_all(["td"])
        # บางครั้งมีช่องว่าง/— ให้คืนเป็น "" ถ้าไม่มี
        val = lambda i: (tds[i].get_text(strip=True) if i < len(tds) else "").replace("\xa0", " ")
        return val(0), val(1), val(2), val(3)

    out = {}

    # 1) Temperature requirement
    tr_t = pick_row("Temperat. requir.")
    if tr_t:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_t)
        out.update({
            "temp_opt_min": opt_min, "temp_opt_max": opt_max,
            "temp_abs_min": abs_min, "temp_abs_max": abs_max,
        })

    # 2) Rainfall (annual)
    tr_r = pick_row("Rainfall (annual)")
    if tr_r:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_r)
        out.update({
            "rain_opt_min": opt_min, "rain_opt_max": opt_max,
            "rain_abs_min": abs_min, "rain_abs_max": abs_max,
        })

    # 3) Soil PH
    tr_p = pick_row("Soil PH")
    if tr_p:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_p)
        out.update({
            "ph_opt_min": opt_min, "ph_opt_max": opt_max,
            "ph_abs_min": abs_min, "ph_abs_max": abs_max,
        })

    return out

def parse_hidden_inputs(soup):
    """สำรอง: ดึงค่า absolute (min/max) จาก hidden inputs ด้านล่างฟอร์ม ถ้าอ่านจากตารางไม่สำเร็จ"""
    hidden = {}
    def val(name):
        el = soup.select_one(f"input[name='{name}']")
        return el.get("value") if el else ""
    hidden["temp_abs_min"] = val("minTemperature")
    hidden["temp_abs_max"] = val("maxTemperature")
    hidden["rain_abs_min"] = val("minRainfall")
    hidden["rain_abs_max"] = val("maxRainfall")
    hidden["ph_abs_min"]   = val("minSoilPh")
    hidden["ph_abs_max"]   = val("maxSoilPh")
    # ไม่มี optimal ใน hidden → ปล่อยว่างไว้
    return hidden

rows = []

try:
    for i, row in df_links.iterrows():
        pid = row["Plant ID"]
        url = row["DataSheetURL"]
        plant_name = row.get("Plant Name", "")
        letter = row.get("Letter", "")

        print(f"📄 [{i+1}/{len(df_links)}] {plant_name} | id={pid}")
        driver.get(url)
        time.sleep(2.0 + random.random())  # รอ DOM

        soup = BeautifulSoup(driver.page_source, "lxml")

        # species ชื่อด้านบน (กันพลาดถ้า Plant Name เดิมว่าง)
        h2 = soup.select_one("div#content h2")
        species = h2.get_text(strip=True) if h2 else plant_name

        eco = parse_ecology_table(soup)
        # ถ้าอ่านตารางไม่ได้หรือได้ไม่ครบ ใช้ hidden inputs สำรองบางค่า
        if not eco or any(eco.get(k, "") == "" for k in ["temp_abs_min","rain_abs_min","ph_abs_min"]):
            eco = {**eco, **parse_hidden_inputs(soup)}

        rows.append({
            "Plant ID": pid,
            "Species": species,
            "Letter": letter,
            "DataSheetURL": url,
            **eco
        })

        # ถนอมเซิร์ฟเวอร์
        time.sleep(0.6 + random.random()*0.6)

finally:
    driver.quit()

df_ecology = pd.DataFrame(rows)

# ทำความสะอาดตัวเลข (ถ้าต้องการเป็น float)
def to_float(x):
    try:
        return float(str(x).replace(",", "").strip())
    except:
        return None

for col in ["temp_opt_min","temp_opt_max","temp_abs_min","temp_abs_max",
            "rain_opt_min","rain_opt_max","rain_abs_min","rain_abs_max",
            "ph_opt_min","ph_opt_max","ph_abs_min","ph_abs_max"]:
    if col in df_ecology.columns:
        df_ecology[col] = df_ecology[col].apply(to_float)

df_ecology.to_csv("ecocrop_ecology_datasheet.csv", index=False)
print("✅ Saved ecocrop_ecology_datasheet.csv")
df_ecology.head(10)

/Users/thanwaratkeratipasuwat/Desktop/dsi314/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


📄 [1/2568] Abelmoschus esculentus | id=289
📄 [2/2568] Abelmoschus manihot | id=290
📄 [3/2568] Abelmoschus moschatus | id=291
📄 [4/2568] Abies alba | id=2550
📄 [5/2568] Abies amabilis | id=196860
📄 [6/2568] Abies balsamea | id=2551
📄 [7/2568] Abies concolor | id=2553
📄 [8/2568] Abies pindrow | id=2558
📄 [9/2568] Abroma augustum | id=2561
📄 [10/2568] Abutilon theophrasti | id=11067
📄 [11/2568] Acacia abyssinica | id=11776
📄 [12/2568] Acacia acradenia | id=2569
📄 [13/2568] Acacia acuminata | id=2570
📄 [14/2568] Acacia ammobia | id=2575
📄 [15/2568] Acacia ampliceps | id=2576
📄 [16/2568] Acacia ancistrocarpa | id=2578
📄 [17/2568] Acacia aneura | id=2579
📄 [18/2568] Acacia angustissima | id=126290
📄 [19/2568] Acacia anticeps | id=2580
📄 [20/2568] Acacia argyrodendron | id=2581
📄 [21/2568] Acacia ataxacantha | id=2582
📄 [22/2568] Acacia aulacocarpa | id=2583
📄 [23/2568] Acacia auriculiformis | id=295
📄 [24/2568] Acacia bidwillii | id=2586
📄 [25/2568] Acacia brachystachya | id=2588
📄 [26/2568]

,Plant ID,Species,Letter,DataSheetURL,temp_opt_min,temp_opt_max,temp_abs_min,temp_abs_max,rain_opt_min,rain_opt_max,rain_abs_min,rain_abs_max,ph_opt_min,ph_opt_max,ph_abs_min,ph_abs_max
0,289,Abelmoschus esculentus,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,20.0,30.0,12.0,35.0,600.0,1200.0,300.0,2500.0,5.5,7.0,4.5,8.7
1,290,Abelmoschus manihot,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,22.0,30.0,16.0,36.0,1200.0,2000.0,1000.0,3000.0,5.5,7.0,5.0,8.0
2,291,Abelmoschus moschatus,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,20.0,30.0,10.0,35.0,1000.0,1400.0,900.0,1500.0,5.5,7.0,5.0,8.0
3,2550,Abies alba,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,196860,Abies amabilis,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,12.0,18.0,6.0,22.0,1500.0,6000.0,1000.0,6650.0,5.0,7.0,4.5,7.5
5,2551,Abies balsamea,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,10.0,18.0,4.0,26.0,750.0,1100.0,400.0,1500.0,5.0,6.5,4.5,7.5
6,2553,Abies concolor,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,10.0,18.0,5.0,22.0,800.0,1200.0,350.0,1900.0,4.5,6.5,4.0,7.0
7,2558,Abies pindrow,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,16.0,24.0,2.0,32.0,1300.0,2000.0,1100.0,2500.0,5.5,6.5,5.0,7.5
8,2561,Abroma augustum,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,26.0,31.0,18.0,36.0,1500.0,3000.0,1300.0,3500.0,5.0,6.0,4.5,6.5
9,11067,Abutilon theophrasti,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,14.0,25.0,10.0,30.0,650.0,2000.0,500.0,3000.0,5.5,6.5,4.9,8.2


In [1]:
# ... (โค้ดช่วง import และฟังก์ชัน parse ทั้งหมดเหมือนเดิม)

import re, time, random, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

BASE = "https://ecocrop.apps.fao.org"

# ---------- 1) เตรียมรายการพืช ----------
df_links = pd.read_csv("ecocrop_links_apps.csv")  # ต้องมี Plant ID หรือ URL อยู่
if "Plant ID" not in df_links.columns:
    # เผื่อไฟล์มีแค่ URL → ดึง id จาก URL
    df_links["Plant ID"] = df_links["URL"].str.extract(r"id=(\d+)")
df_links = df_links.dropna(subset=["Plant ID"]).copy()
df_links["Plant ID"] = df_links["Plant ID"].astype(str)
df_links["DataSheetURL"] = BASE + "/ecocrop/srv/en/dataSheet?id=" + df_links["Plant ID"]

# ---------- 2) ตั้งค่า Selenium ----------
opts = webdriver.ChromeOptions()
# opts.add_argument("--headless=new")   # เปิดถ้าต้องการรันแบบไม่เปิดหน้าจอ
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--window-size=1366,1000")
opts.add_argument("--lang=en-US")
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def parse_ecology_table(soup):
    """คืน dict ของ 3 แถว Ecology ที่ต้องการ: Temperature, Rainfall, Soil PH
       เก็บ Optimal Min/Max และ Absolute Min/Max"""
    # หา table ที่หัวคอลัมน์เขียนว่า Ecology
    target = None
    for tbl in soup.select("div#content table"):
        th = tbl.find("th")
        if th and "Ecology" in th.get_text(strip=True):
            target = tbl
            break
    if target is None:
        return {}

    def pick_row(label):
        # หาแถวที่หัวคอลัมน์ (th) เท่ากับ label
        row = target.find("tr", string=lambda x: False)
        for tr in target.find_all("tr"):
            head = tr.find("th")
            if head and head.get_text(strip=True) == label:
                return tr
        return None

    def get_vals(tr):
        # โครงสร้างคอลัมน์: th | Opt Min | Opt Max | Abs Min | Abs Max | (อีกฝั่งเป็น Soil อื่น ๆ)
        # เราจะอ่านแค่ 4 ช่องแรกหลัง th
        tds = tr.find_all(["td"])
        # บางครั้งมีช่องว่าง/— ให้คืนเป็น "" ถ้าไม่มี
        val = lambda i: (tds[i].get_text(strip=True) if i < len(tds) else "").replace("\xa0", " ")
        return val(0), val(1), val(2), val(3)

    out = {}

    # 1) Temperature requirement
    tr_t = pick_row("Temperat. requir.")
    if tr_t:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_t)
        out.update({
            "temp_opt_min": opt_min, "temp_opt_max": opt_max,
            "temp_abs_min": abs_min, "temp_abs_max": abs_max,
        })

    # 2) Rainfall (annual)
    tr_r = pick_row("Rainfall (annual)")
    if tr_r:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_r)
        out.update({
            "rain_opt_min": opt_min, "rain_opt_max": opt_max,
            "rain_abs_min": abs_min, "rain_abs_max": abs_max,
        })

    # 3) Soil PH
    tr_p = pick_row("Soil PH")
    if tr_p:
        opt_min, opt_max, abs_min, abs_max = get_vals(tr_p)
        out.update({
            "ph_opt_min": opt_min, "ph_opt_max": opt_max,
            "ph_abs_min": abs_min, "ph_abs_max": abs_max,
        })

    return out

def parse_hidden_inputs(soup):
    """สำรอง: ดึงค่า absolute (min/max) จาก hidden inputs ด้านล่างฟอร์ม ถ้าอ่านจากตารางไม่สำเร็จ"""
    hidden = {}
    def val(name):
        el = soup.select_one(f"input[name='{name}']")
        return el.get("value") if el else ""
    hidden["temp_abs_min"] = val("minTemperature")
    hidden["temp_abs_max"] = val("maxTemperature")
    hidden["rain_abs_min"] = val("minRainfall")
    hidden["rain_abs_max"] = val("maxRainfall")
    hidden["ph_abs_min"]   = val("minSoilPh")
    hidden["ph_abs_max"]   = val("maxSoilPh")
    # ไม่มี optimal ใน hidden → ปล่อยว่างไว้
    return hidden

rows = []

try:
    # ➤ โหลดแค่ 10 อันแรกเพื่อทดสอบ
    for i, row in df_links.iloc[:10].iterrows():
        pid = row["Plant ID"]
        url = row["DataSheetURL"]
        plant_name = row.get("Plant Name", "")
        letter = row.get("Letter", "")

        print(f"📄 [{i+1}/10] {plant_name} | id={pid}")
        driver.get(url)
        time.sleep(2.0 + random.random())  # รอ DOM โหลด

        soup = BeautifulSoup(driver.page_source, "lxml")

        # species ชื่อด้านบน (กันพลาดถ้า Plant Name เดิมว่าง)
        h2 = soup.select_one("div#content h2")
        species = h2.get_text(strip=True) if h2 else plant_name

        eco = parse_ecology_table(soup)
        # ถ้าอ่านตารางไม่ได้หรือได้ไม่ครบ ใช้ hidden inputs สำรองบางค่า
        if not eco or any(eco.get(k, "") == "" for k in ["temp_abs_min","rain_abs_min","ph_abs_min"]):
            eco = {**eco, **parse_hidden_inputs(soup)}

        rows.append({
            "Plant ID": pid,
            "Species": species,
            "Letter": letter,
            "DataSheetURL": url,
            **eco
        })

        # ถนอมเซิร์ฟเวอร์
        time.sleep(0.6 + random.random()*0.6)

finally:
    driver.quit()

df_ecology = pd.DataFrame(rows)

# ทำความสะอาดตัวเลข (ถ้าต้องการเป็น float)
def to_float(x):
    try:
        return float(str(x).replace(",", "").strip())
    except:
        return None

for col in ["temp_opt_min","temp_opt_max","temp_abs_min","temp_abs_max",
            "rain_opt_min","rain_opt_max","rain_abs_min","rain_abs_max",
            "ph_opt_min","ph_opt_max","ph_abs_min","ph_abs_max"]:
    if col in df_ecology.columns:
        df_ecology[col] = df_ecology[col].apply(to_float)

df_ecology.to_csv("ecocrop_ecology_datasheet_sample10.csv", index=False)
print("✅ Saved ecocrop_ecology_datasheet_sample10.csv")
df_ecology.head(10)

/Users/thanwaratkeratipasuwat/Desktop/dsi314/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


📄 [1/10] Abelmoschus esculentus | id=289
📄 [2/10] Abelmoschus manihot | id=290
📄 [3/10] Abelmoschus moschatus | id=291
📄 [4/10] Abies alba | id=2550
📄 [5/10] Abies amabilis | id=196860
📄 [6/10] Abies balsamea | id=2551
📄 [7/10] Abies concolor | id=2553
📄 [8/10] Abies pindrow | id=2558
📄 [9/10] Abroma augustum | id=2561
📄 [10/10] Abutilon theophrasti | id=11067
✅ Saved ecocrop_ecology_datasheet_sample10.csv


,Plant ID,Species,Letter,DataSheetURL,temp_opt_min,temp_opt_max,temp_abs_min,temp_abs_max,rain_opt_min,rain_opt_max,rain_abs_min,rain_abs_max,ph_opt_min,ph_opt_max,ph_abs_min,ph_abs_max
0,289,Abelmoschus esculentus,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,20.0,30.0,12.0,35.0,600.0,1200.0,300.0,2500.0,5.5,7.0,4.5,8.7
1,290,Abelmoschus manihot,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,22.0,30.0,16.0,36.0,1200.0,2000.0,1000.0,3000.0,5.5,7.0,5.0,8.0
2,291,Abelmoschus moschatus,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,20.0,30.0,10.0,35.0,1000.0,1400.0,900.0,1500.0,5.5,7.0,5.0,8.0
3,2550,Abies alba,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,196860,Abies amabilis,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,12.0,18.0,6.0,22.0,1500.0,6000.0,1000.0,6650.0,5.0,7.0,4.5,7.5
5,2551,Abies balsamea,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,10.0,18.0,4.0,26.0,750.0,1100.0,400.0,1500.0,5.0,6.5,4.5,7.5
6,2553,Abies concolor,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,10.0,18.0,5.0,22.0,800.0,1200.0,350.0,1900.0,4.5,6.5,4.0,7.0
7,2558,Abies pindrow,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,16.0,24.0,2.0,32.0,1300.0,2000.0,1100.0,2500.0,5.5,6.5,5.0,7.5
8,2561,Abroma augustum,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,26.0,31.0,18.0,36.0,1500.0,3000.0,1300.0,3500.0,5.0,6.0,4.5,6.5
9,11067,Abutilon theophrasti,A,https://ecocrop.apps.fao.org/ecocrop/srv/en/da...,14.0,25.0,10.0,30.0,650.0,2000.0,500.0,3000.0,5.5,6.5,4.9,8.2
